In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
from scipy.special import erf, erfinv
import gmplot

from matplotlib.lines import Line2D

In [2]:
crises_df = pd.read_csv('data/african_crises.csv')
crises_df = crises_df.replace({'banking_crisis': {'crisis': 1, 'no_crisis': 0}})
print(crises_df.isnull().sum())
print()
for index, col in enumerate(crises_df.columns):
    print(index, col)

case                               0
cc3                                0
country                            0
year                               0
systemic_crisis                    0
exch_usd                           0
domestic_debt_in_default           0
sovereign_external_debt_default    0
gdp_weighted_default               0
inflation_annual_cpi               0
independence                       0
currency_crises                    0
inflation_crises                   0
banking_crisis                     0
dtype: int64

0 case
1 cc3
2 country
3 year
4 systemic_crisis
5 exch_usd
6 domestic_debt_in_default
7 sovereign_external_debt_default
8 gdp_weighted_default
9 inflation_annual_cpi
10 independence
11 currency_crises
12 inflation_crises
13 banking_crisis


In [10]:
zaf = (-30.28449, 23.965389)

lats = [zaf[0]]
longs = [zaf[1]]

gmap.heatmap(lats, longs)
gmap = gmplot.GoogleMapPlotter(5.777596, 17.037116, 1.5)


In [11]:
gmap.draw("my_heatmap.html")

.

.

.

.

.

.

.


In [73]:
import folium 
import json
import copy

countriesString = 'world-countries.json'
file = open(countriesString).read()
countries = json.loads(file)


In [85]:
def filterCountries(countries, countrycodes):
    indexes = []
    accepted = []
    for i in range(len(countries['features'])):
        if countries['features'][i]['id'] not in countrycodes:
            indexes += [i]
#             print(countries['features'][i]['id'], countries['features'][i]['properties']['name'])
        else:
            accepted += [countries['features'][i]['id']]
            
    newcountries = copy.deepcopy(countries)
    indexes.sort(reverse=True)
    for index in indexes:
        newcountries['features'].pop(index)
    return newcountries, accepted

countrycodes = {'France': ['BEN', 'BFA', 'CAF', 'CIV', 'CMR', 'COM', 'GAB', 'GIN', 'MDG','MLI', 'MRT', 'NER', 'SEN', 'TCD', 'TGO', 'MAR', 'DZA', 'TUN'], 'GB': ['BWA', 'GHA', 'GMB', 'KEN', 'LSO', 'MUS', 'MWI', 'NGA', 'SDN', 'SLE', 'SSD', 'SWZ', 'SYC', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE','EGY'], 'Belgium': ['BDI', 'COD', 'COG', 'RWA'], 'Italy': ['ERI', 'SOM'], 'Portugal': ['AGO', 'CPV', 'GNB', 'MOZ', 'STP'], 'Spain': ['GNQ'], 'Ind': ['ETH', 'LBR']}
showcountries = [code for code in countrycodes['France']] + [code for code in countrycodes['GB']]

filteredCountries, accepted = filterCountries(countries, showcountries)
# print(len(showcountries))
# print(len(filteredCountries['features']))
# for country in showcountries:
#     if country not in accepted:
#         print(country)


In [108]:
m = folium.Map(location=[2,17], zoom_start=3, zoomControl=False)
m.choropleth(
    geo_data=filteredCountries, 
    name='choropleth',
    zoom_on_click=False,
    zoomControl=False)
m.save('heatmap.html')
m